In [5]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name

conf = (SparkConf().setMaster("k8s://https://192.168.219.100:6443") # Your master address name
        .set("spark.kubernetes.container.image", "joron1827/pyspark:v2") # Spark image name
        .set("spark.driver.port", "2222") # Needs to match svc
        .set("spark.driver.blockManager.port", "7777")
        .set("spark.driver.host", "driver-service.jupyterhub.svc.cluster.local") # Needs to match svc
        .set("spark.driver.bindAddress", "0.0.0.0")
        .set("spark.kubernetes.namespace", "spark")
        .set("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")
        .set("spark.kubernetes.authenticate.serviceAccountName", "spark")
        .set("spark.executor.instances", "8")
        .set("spark.kubernetes.container.image.pullPolicy", "IfNotPresent")
        .set("spark.app.name", "joronSpark")
        .set("spark.executor.cores", "2")
        .set("spark.executor.memory", "12g"))

In [6]:

spark = SparkSession.builder.config(conf=conf).getOrCreate()

import time

time.sleep(10)

/home/jovyan/.local/lib/python3.9/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found


23/06/01 20:08:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [18]:
filtered_df = tokenized_df.filter((col("New_Column") == 0) | (col("New_Column") == 2))
tokenized_df = filtered_df.withColumn("New_Column", when(col("New_Column") == 2, 1).otherwise(col("New_Column")))

tokenized_df.show()

+------+----------+------------------------------------+----------+------------------------------+
|  code|      date|                               title|New_Column|                        tokens|
+------+----------+------------------------------------+----------+------------------------------+
|015760|2022-05-30|    이리저리 아무리 생각을 해 보아도|         1|[이리저리, 아무리, 생각, 해...|
|015760|2022-05-30|    이리저리 아무리 생각을 해 보아도|         1|[이리저리, 아무리, 생각, 해...|
|015760|2022-05-30|                  최순실때 만 갔으니|         1|    [최, 순, 실때, 만, 갔으니]|
|015760|2022-05-30|어디에도 전기요금 동결 이야기 없는데|         1|[에도, 전기요금, 동결, 이야...|
|015760|2022-05-30|                    매출은 오르는데 |         1|          [매출, 은, 오르는데]|
|015760|2022-05-30|                         당분간 하락|         1|                [당분간, 하락]|
|015760|2022-05-30|         죄인이처럼 나라 망치지 말자|         1|  [죄인, 처럼, 나라, 망치, ...|
|015760|2022-05-30|              공매도 배 증가전날대비|         1|[공매도, 배, 증가, 전날, 대비]|
|015760|2022-05-30|                   오늘 용돈 받았다 |         1|     

In [19]:
tokenized_df = tokenized_df.dropDuplicates(["title","tokens"])
tokenized_df.show()

+------+----------+-------------------------------+----------+------------------------------+
|  code|      date|                          title|New_Column|                        tokens|
+------+----------+-------------------------------+----------+------------------------------+
|027740|2019-05-07|                         조천억|         1|                      [조천억]|
|007160|2017-12-27|                    니   니  니|         0|                  [니, 니, 니]|
|066570|2020-04-13|                           매동|         1|                          [동]|
|028050|2021-09-24|             숨겨놓고 조질나...|         1|  [숨겨놓고, 조질, 나게, 판다]|
|026890|2018-10-29|                 공매도취재요청|         1|          [공매도, 취재, 요청]|
|007570|2021-03-11|          문재인 대통령 께서...|         0|[문재인, 대통령, 께서, 수출...|
|042670|2021-02-08|              반복해서 말해보니|         1|      [반복, 해서, 말, 해보니]|
|002210|2018-09-12|           오늘 장분석   과 ...|         1|    [오늘, 장, 분석, 낼, 예측]|
|002210|2018-08-31|       동성제약  장분석  입니다|         1|  [동성제약,

In [13]:
tokenized_df.count()

769381

In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Word2Vec
'''
먼저 HashingTF를 사용하여 토큰화된 데이터를 TF(Term Frequency) 벡터로 변환합니다.
그런 다음 IDF를 사용하여 TF 벡터를 TF-IDF(Inverse Document Frequency) 벡터로 변환합니다.
TF-IDF 벡터화된 데이터를 Word2Vec 모델에 입력으로 사용하여 학습시킵니다.

이렇게 하면 TF-IDF와 Word2Vec을 조합하여 문맥과 단어의 의미를 모두 반영하는 워드 임베딩을 생성할 수 있습니다.
옵션을 사용하는 것보다는 이러한 조합을 통해 원하는 기능을 구현할 수 있습니다.
'''

# TF-IDF 벡터화를 위한 단계
hashingTF = HashingTF(inputCol="tokens", outputCol="tf")
tf_features = hashingTF.transform(tokenized_df)

idf = IDF(inputCol="tf", outputCol="tfidf")
idf_model = idf.fit(tf_features)
tfidf_features = idf_model.transform(tf_features)

tfidf_features.show()

idf_model.save("hdfs://192.168.219.121:9000/crawling_stock_text_merge/two_class_idf_model")


In [ ]:
# Word2Vec 모델 정의 및 학습
word2Vec = Word2Vec(vectorSize=100, inputCol="tokens", outputCol="word2vec")
word2Vec_model = word2Vec.fit(tfidf_features)
word2Vec_features = word2Vec_model.transform(tfidf_features)
word2Vec_features.show()
word2Vec_model.save("hdfs://192.168.219.121:9000/crawling_stock_text_merge/two_class_word2Vec_model")

save_path = "hdfs://192.168.219.121:9000/crawling_stock_text_merge/two_class_text_vectorized.parquet"

word2Vec_features.write.parquet(save_path)

In [7]:
df = spark.read.parquet("hdfs://192.168.219.121:9000/crawling_stock_text_merge/two_class_text_vectorized.parquet")
df.show()

+------+----------+-------------------------------+----------+-----------------------------+--------------------+--------------------+--------------------+
|  code|      date|                          title|New_Column|                       tokens|                  tf|               tfidf|            word2vec|
+------+----------+-------------------------------+----------+-----------------------------+--------------------+--------------------+--------------------+
|035720|2017-09-26|                               |         1|                           []|      (262144,[],[])|      (262144,[],[])|         (100,[],[])|
|003550|2022-03-16|                      주주에...|         0|   [주주, 원한, 잇는건, 가요]|(262144,[51217,78...|(262144,[51217,78...|[0.01652339892461...|
|323410|2021-08-09|                           이이|         1|                       [이이]|(262144,[104757],...|(262144,[104757],...|[0.03197551518678...|
|007700|2020-06-26|                        와 소름|         0|                     

In [16]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# 학습 데이터와 테스트 데이터 분할 (70% 학습, 30% 테스트)
train_data, test_data = df.randomSplit([0.7, 0.3], seed=42)

# 로지스틱 회귀 모델 초기화
lr = LogisticRegression(labelCol="New_Column", featuresCol="word2vec")

# 모델 학습
model = lr.fit(train_data)

# 테스트 데이터에 대한 예측
predictions = model.transform(test_data)

# 예측 결과 확인
predictions.select("New_Column", "prediction", "probability").show()

# 이진 분류 평가 지표 계산 (예시로는 Area Under ROC를 사용)
evaluator = BinaryClassificationEvaluator(labelCol="New_Column")
auc = evaluator.evaluate(predictions)

predictions.show()
print("Area Under ROC:", auc)


+----------+----------+--------------------+
|New_Column|prediction|         probability|
+----------+----------+--------------------+
|         1|       1.0|[0.38771389493341...|
|         1|       1.0|[0.30762478700942...|
|         1|       1.0|[0.29564924712820...|
|         1|       1.0|[0.36427519530033...|
|         1|       1.0|[0.32625469297149...|
|         1|       1.0|[0.25081950628528...|
|         1|       1.0|[0.34254753549562...|
|         0|       1.0|[0.27334411151194...|
|         0|       1.0|[0.39324331467579...|
|         1|       1.0|[0.37901248233718...|
|         1|       1.0|[0.33579695574205...|
|         0|       1.0|[0.25930828593347...|
|         0|       1.0|[0.39206421910840...|
|         0|       1.0|[0.38697373511627...|
|         0|       1.0|[0.37890035477195...|
|         0|       1.0|[0.36482174196766...|
|         0|       1.0|[0.37947296414590...|
|         0|       1.0|[0.35479864220121...|
|         1|       1.0|[0.39940664937266...|
|         

+------+----------+------------------------------------+----------+------------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  code|      date|                               title|New_Column|                        tokens|                  tf|               tfidf|            word2vec|            features|       rawPrediction|         probability|prediction|
+------+----------+------------------------------------+----------+------------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|000020|2020-06-12|   요기 고수가 어딧냐 ㄱ 초보들뿐...|         1|  [요기, 고수, 어딧, 냐, ㄱ...|(262144,[43344,77...|(262144,[43344,77...|[-0.0527202452596...|[-0.0527202452596...|[-0.4569319817812...|[0.38771389493341...|       1.0|
|000020|2020-07-29|                   동화약품과 제테마|         1|              [동화약

In [15]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline

# RandomForestClassifier 모델을 생성합니다
rf = RandomForestClassifier(labelCol="New_Column", featuresCol="word2vec")

# 파이프라인을 생성합니다
pipeline = Pipeline(stages=[rf])

# 모델을 학습시킵니다
model = pipeline.fit(train_data)  # trainData는 학습에 사용할 데이터 프레임입니다

# 학습된 모델을 사용하여 예측을 수행합니다
predictions = model.transform(test_data)  # testData는 예측에 사용할 데이터 프레임입니다

# 예측 결과 확인
predictions.select("New_Column", "prediction", "probability").show()

+----------+----------+--------------------+
|New_Column|prediction|         probability|
+----------+----------+--------------------+
|         1|       1.0|[0.34390539966221...|
|         1|       1.0|[0.35685039350669...|
|         1|       1.0|[0.33849279022470...|
|         1|       1.0|[0.34827823429516...|
|         1|       1.0|[0.34506017959558...|
|         1|       1.0|[0.34755194720732...|
|         1|       1.0|[0.34110266046455...|
|         0|       1.0|[0.34452171239640...|
|         0|       1.0|[0.34762383652253...|
|         1|       1.0|[0.34807300949824...|
|         1|       1.0|[0.34419556936462...|
|         0|       1.0|[0.33490293486148...|
|         0|       1.0|[0.34934310765584...|
|         0|       1.0|[0.34964418771949...|
|         0|       1.0|[0.33852623756317...|
|         0|       1.0|[0.35042475481537...|
|         0|       1.0|[0.34761696782093...|
|         0|       1.0|[0.34572773519702...|
|         1|       1.0|[0.35475344484117...|
|         

In [12]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# MulticlassClassificationEvaluator를 사용하여 정확도를 계산합니다
evaluator = MulticlassClassificationEvaluator(labelCol="New_Column", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print("Accuracy:", accuracy)


Accuracy: 0.6551456158935797


In [13]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# MulticlassClassificationEvaluator를 사용하여 정밀도, 재현율, F1 스코어를 계산합니다
evaluator = MulticlassClassificationEvaluator(labelCol="New_Column", predictionCol="prediction")

# 정밀도 계산
precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})

# 재현율 계산
recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})

# F1 스코어 계산
f1_score = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


Precision: 0.42921577802457783
Recall: 0.6551456158935797
F1 Score: 0.5186441288343719


In [14]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# 의사결정트리 모델 생성
dt = DecisionTreeClassifier(labelCol="New_Column", featuresCol="word2vec")

# 모델 훈련
dt_model = dt.fit(train_data)

# 테스트 데이터에 대한 예측
dt_predictions = dt_model.transform(test_data)

# 평가 지표 계산을 위한 MulticlassClassificationEvaluator 생성
evaluator = MulticlassClassificationEvaluator(labelCol="New_Column", predictionCol="prediction")

# 정확도 계산
accuracy = evaluator.evaluate(dt_predictions, {evaluator.metricName: "accuracy"})

# 정밀도 계산
precision = evaluator.evaluate(dt_predictions, {evaluator.metricName: "weightedPrecision"})

# 재현율 계산
recall = evaluator.evaluate(dt_predictions, {evaluator.metricName: "weightedRecall"})

# F1 스코어 계산
f1_score = evaluator.evaluate(dt_predictions, {evaluator.metricName: "f1"})

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


Accuracy: 0.6551456158935797
Precision: 0.42921577802457783
Recall: 0.6551456158935797
F1 Score: 0.5186441288343719
